# Output Keyword

SQL Server comes across as a very closed system, providing minimal feedback for what could well be complex and wide ranging updates.

For example, we can code and run a DELETE statement against a table in the database and receive nothing more than 'X rows affected' in the SQL Server Management Studio messages tab.

This can be frightening if you are unsure that the 'X rows' matches the number you expected to remove as there is no way back once the action has been taken.

Luckily the OUTPUT keyword can help out here as it allows us to access special internal tables inside SQL Server which track a 'before and after' view of our data.

When used correctly it can be utitlised as an audit mechanism or even a full log of changes, providing the ability to rollback if something goes wrong.

#### Output with Insert

For the following demonstration we shall use a very basic employee table designed to hold only 3 columns; employeeID, firstName, and lastName.

It looks as follows:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

Our first port of call is to INSERT a record of data into this table.

However, if you'll notice from the table definition, the temporary table has an IDENTITY column and therefore when we INSERT into the table we will only pass in a first and last name.

The downside here is that we wish to know the employeeID of our new record.

We COULD do this as follows:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

insert into #employee(firstName, lastName)
select 'John', 'Smith';

select employeeID
from #employee
where firstName = 'John'
and lastName = 'Smith';

This works just fine.  But what if there were multiple John Smiths in the employee roster?  Also, we could have incorrectly entered the name in the INSERT and correctly spelt it in retrieval, therefore returning nothing.

Obviously these are obscure and forced examples, but they do serve a purpose in explanation.

This is where the OUTPUT keyword can come into play.

In SQL Server, when a modification transaction takes place, there are two internal tables created; deleted and inserted.  These are best thought of as before and after, respectively.

These tables are not visible to the user and are only accessible during the transaction itself.  The moment the INSERT has finished, the tables are destroyed.

OUTPUT allows us to interact with these tables during a transaction, therefore giving us access to the before and after views of the modification.

To use our example above, we wish to INSERT a record into our table and with to be told, by SQL Server, what the employeeID is of the INSERTed record.  Ie. we need access to the 'after' table.

Let's look at this with an example:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

insert into #employee(firstName, lastName)
output inserted.employeeID  -- 'output the employeeID from the inserted table'
select 'John', 'Smith';

Executing this code shows us the exact output we require.

We can also return any of the other columns in order to validate that we did, in fact, spell the name correctly and did not mess up our INSERT:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

insert into #employee(firstName, lastName)
output inserted.employeeID, inserted.firstName, inserted.lastName
select 'John', 'Smith';

#### Output with Update

As you can imagine, this is exactly the same principle for an UPDATE, except that with an UPDATE we can not only see the 'after', but also the 'before' record.

Using the same temporary table we will INSERT a single record into our table but, this time, we will then perform an update on the data, changing the firstName from 'John' to 'Fred'.

We now wish to see the effects of this update both before and after the change as part of our statement.  To do this we will need to leverage both the deleted and inserted tables and will alias our columns accordingly:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

insert into #employee(firstName, lastName)
select 'John', 'Smith';

-- Perform the UPDATE, returning both deleted and inserted records
update e
set e.firstname = 'Fred'
output deleted.employeeID as previousEmployeeID,
        deleted.firstName as previousFirstName,
        deleted.lastName as previousLastName,
        inserted.employeeID as newEmployeeID,
        inserted.firstName as newFirstName,
        inserted.lastName as newLastName
from #employee as e
where e.firstName = 'John'
and e.lastName = 'Smith';

Hence this is a great way to keep an audit on your changes.

#### Output with Delete

Finally we look at the OUTPUT keyword used alongside DELETE.

As you can imagine from the above, the syntax and the usage is the same with the exception that we only have access to the deleted table, for obvious reasons.

Therefore we will use the same temporary table but this time we will INSERT a record and then DELETE it but, alongside the DELETE, we will output the information we are deleting:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

insert into #employee(firstName, lastName)
select 'John', 'Smith';

-- Perform the DELETE, returning the deleted records
delete e
output deleted.employeeID as deletedEmployeeID,
        deleted.firstName as deletedFirstName,
        deleted.lastName as deletedLastName
from #employee as e
where e.firstName = 'John'
and e.lastName = 'Smith';

However, at this stage you may well be thinking that this is great if you manually enter your changes, but what happens if an application is making them and you aren't there to see the output?

This isn't a problem as OUTPUT will allow you to log this data to other tables, providing a long term audit of your modifications for later analysis.

#### Output with Into

For this next example we will use two temporary tables; #employee, and #employeeAudit.  The code for these is as follows:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

drop table if exists #employeeAudit;

create table #employeeAudit
(
    employeeID int,
    previousFirstName varchar(25),
    previousLastName varchar(35),
    newFirstName varchar(25),
    newLastName varchar(35),
    auditDate datetime default(current_timestamp)
);

As you can see we have created an audit table which has a default on the auditDate so that it can automatically track the date and time that any change was made without us having to worry about coding it.

Returning to our knowledge that SQL is a declarative language let's see if we can imagine how this might go as we wish to 'INSERT INTO our table and OUTPUT this data INTO this table'.

Hence we expect the syntax to be as follows:

<code>INSERT INTO <i>tableName</i><br/>
OUTPUT <i>inserted and deleted columns</i><br/>
INTO <i>archiveTable</i><br/>
SELECT <i>columns</i><br/>
FROM <i>table</i><br/>
WHERE <i>column = something</i></code>

Therefore let's use this with an INSERT so that we can get a feel for the syntax of using OUTPUT with INTO:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

drop table if exists #employeeAudit;

create table #employeeAudit
(
    employeeID int,
    previousFirstName varchar(25),
    previousLastName varchar(35),
    newFirstName varchar(25),
    newLastName varchar(35),
    auditDate datetime default(current_timestamp)
);

-- INSERT a record, logging this out to our archive table
insert into #employee(firstName, lastName)
output inserted.employeeID, inserted.firstName, inserted.lastName
into #employeeAudit(employeeID, newFirstName, newLastName)
select 'John', 'Smith';

-- Validate results
select *
from #employee;

select *
from #employeeAudit;

This is exactly what we were looking to achieve.

DELETE works in the same way, therefore we will INSERT a record and then DELETE it, logging both changes:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

drop table if exists #employeeAudit;

create table #employeeAudit
(
    employeeID int,
    previousFirstName varchar(25),
    previousLastName varchar(35),
    newFirstName varchar(25),
    newLastName varchar(35),
    auditDate datetime default(current_timestamp)
);

-- INSERT a record, logging this out to our archive table
insert into #employee(firstName, lastName)
output inserted.employeeID, inserted.firstName, inserted.lastName
into #employeeAudit(employeeID, newFirstName, newLastName)
select 'John', 'Smith';

-- DELETE a record, logging this out to our archive table
delete e
output deleted.employeeID, deleted.firstName, deleted.lastName
into #employeeAudit(employeeID, previousFirstName, previousLastName)
from #employee as e
where e.firstName = 'John'
and e.lastName = 'Smith';

-- Validate results
select *
from #employee;

select *
from #employeeAudit;

This shows us that we have no records remaining in our #employee table, which is exactly as we would expect.

It also shows that our #employeeAudit has both our INSERT and DELETE with dates and times that they took place, providing the audit we require.

Finally, to complete the set, we shall INSERT a record, UPDATE the firstName from 'John' to 'Fred' and then DELETE the record, logging everything as we proceed:

In [ ]:
drop table if exists #employee;

create table #employee
(
    employeeID int identity(1234, 5),
    firstName varchar(25),
    lastName varchar(35)
);

drop table if exists #employeeAudit;

create table #employeeAudit
(
    employeeID int,
    previousFirstName varchar(25),
    previousLastName varchar(35),
    newFirstName varchar(25),
    newLastName varchar(35),
    auditDate datetime default(current_timestamp)
);

-- INSERT a record, logging this out to our archive table
insert into #employee(firstName, lastName)
output inserted.employeeID, inserted.firstName, inserted.lastName
into #employeeAudit(employeeID, newFirstName, newLastName)
select 'John', 'Smith';

-- UPDATE a record, logging this out to our archive table
update e
set e.firstName = 'Fred'
output inserted.employeeID, deleted.firstName, deleted.lastName,
        inserted.firstName, inserted.lastName
into #employeeAudit
(
    employeeID, previousFirstName, previousLastName,
    newFirstName, newLastName
)
from #employee as e
where e.firstName = 'John'
and e.lastName = 'Smith';

-- DELETE a record, logging this out to our archive table
delete e
output deleted.employeeID, deleted.firstName, deleted.lastName
into #employeeAudit(employeeID, previousFirstName, previousLastName)
from #employee as e
where e.firstName = 'Fred'
and e.lastName = 'Smith';

-- Validate results
select *
from #employee;

select *
from #employeeAudit;

Once again we are left with no records in our #employee table, yet we now have three records in our #employeeAudit which show a full trail of everything that has happened, leaving no ambuguity or guesswork as to what changes happened and how we arrived at the current state.